# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [15]:
import pandas as pd
import numpy as np
import re
import warnings

warnings.filterwarnings('ignore') 

# step 1 Reading and Combining Files

file1_url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv'
file2_url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv'
file3_url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv'

try:
    df1 = pd.read_csv(file1_url)
    df2 = pd.read_csv(file2_url)
    df3 = pd.read_csv(file3_url)
    
    # Combine the three DataFrames
    raw_data_combined = pd.concat([df1, df2, df3], ignore_index=True)
    print(f"   Success! Combined total rows: {raw_data_combined.shape[0]}")
except Exception as e:
    print(f"   Error reading/combining files: {e}")


# step 2 Standardizing Column Names

def standardize_column_names(df):
    cols = df.columns
    # Lowercase, replace spaces/hyphens/dots with underscores, and remove residual non-word characters
    cols = [re.sub(r'[^\w]', '', re.sub(r'[\s\-\.]', '_', col.lower())) for col in cols]
    df.columns = cols
    return df

combined_data_clean_cols = standardize_column_names(raw_data_combined.copy())
print(f"   First 5 new columns: {combined_data_clean_cols.columns[:5].tolist()}")


# STEP 3: Cleaning Text Data (Lowercasing & Standardizing)

def clean_text_data(df):
    object_cols = df.select_dtypes(include=['object']).columns
    
    for col in object_cols:
        series_to_clean = df[col]
        
        # 1. Convert the Series values to string type.
        series_to_clean = series_to_clean.astype(str)
        
        # 2. **FINAL FIX**: Use .apply(lambda x) and force str() conversion on the element 'x'
        # This guarantees 'x' is a string before calling .lower()
        series_to_clean = series_to_clean.apply(lambda x: str(x).lower().strip())
        
        # 3. Replace common string representations of NaN with actual NaN
        series_to_clean.replace(['nan', 'na', 'n/a', '-'], np.nan, inplace=True)
        
        # 4. Assign the cleaned Series back to the DataFrame
        df[col] = series_to_clean
    
    return df

# Run the fixed function
combined_data_clean_text = clean_text_data(combined_data_clean_cols.copy())
print("Text cleaning done.")


## STEP 4: Converting Numeric Data Types...

def clean_and_convert_numerics(df):
    numeric_cols_to_clean = [
        'customer_lifetime_value', 
        'income', 
        'monthly_premium_auto', 
        'total_claim_amount',
        'number_of_open_complaints',
        'number_of_policies'
    ]
    
    for col in numeric_cols_to_clean:
        if col in df.columns:
            # Remove '$', ',', '%', and other symbols
            df[col] = df[col].astype(str).str.replace(r'[$,%]', '', regex=True)
            
            # Convert to numeric, coercing errors to NaN
            df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

combined_data_clean_numerics = clean_and_convert_numerics(combined_data_clean_text.copy())
print(f"   Numeric columns dtypes: {combined_data_clean_numerics[['income', 'total_claim_amount']].dtypes.to_dict()}")


# STEP 5: Handling Missing Values (Imputation)

def handle_missing_values(df):
    numeric_cols = df.select_dtypes(include=np.number).columns
    object_cols = df.select_dtypes(include='object').columns
    
    # Impute Numeric Columns with the Mean
    for col in numeric_cols:
        df[col].fillna(df[col].mean(), inplace=True)
        
    # Impute Categorical Columns with the Mode
    for col in object_cols:
        try:
            # Mode can fail if all values are NaN
            mode_val = df[col].mode().iloc[0] 
            df[col].fillna(mode_val, inplace=True)
        except IndexError:
            # Fallback for columns entirely filled with NaNs
            df[col].fillna('UNKNOWN', inplace=True) 
        
    return df

combined_data_final_clean = handle_missing_values(combined_data_clean_numerics.copy())
total_missing = combined_data_final_clean.isnull().sum().sum()

print(f"   Imputation complete. Total missing values remaining: {total_missing}")

# Final verification
print("Final Clean Data Sample:")
print(combined_data_final_clean.head())

--- Starting Challenge 1: Combining & Cleaning Data ---

[1/5] Reading and Combining Files...
   Success! Combined total rows: 12074

[2/5] Standardizing Column Names (Snake Case)...
   First 5 new columns: ['customer', 'st', 'gender', 'education', 'customer_lifetime_value']

[3/5] Cleaning Text Data (Lowercasing & Standardizing)...
   Text cleaning complete.

[4/5] Converting Numeric Data Types...
   Numeric columns dtypes: {'income': dtype('float64'), 'total_claim_amount': dtype('float64')}

[5/5] Handling Missing Values (Imputation)...
   Imputation complete. Total missing values remaining: 0
--- Pipeline Complete! Data is ready for analysis. ---

Final Clean Data Sample:
  customer          st                                             gender  \
0  rb50392  washington  0        0        nan\n1          f\n2        ...   
1  qz44356     arizona  0        0        nan\n1          f\n2        ...   
2  ai49188      nevada  0        0        nan\n1          f\n2        ...   
3  ww632

In [16]:
import pandas as pd

output_file_path = 'combined_data_final_clean.csv'


try:
    combined_data_final_clean.to_csv(output_file_path, index=False)
    print(f"✅ Success! Your clean data has been saved to: {output_file_path}")

    
except NameError:
    print("\nERROR: 'combined_data_final_clean' is not defined.")
    print("Please ensure you run all five steps of the consolidated pipeline (with the final fix for Step 3) before running this cell.")
except Exception as e:
    print(f"\nAn error occurred while saving the file: {e}")

✅ Success! Your clean data has been saved to: combined_data_final_clean.csv


# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

In [17]:
import pandas as pd
import numpy as np

DATA_URL = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv"

try:
    df = pd.read_csv(DATA_URL)
    print("'marketing_customer_analysis_clean.csv' loaded.")
    print("-" * 50)
except Exception as e:
    print(f"Error loading dataset: {e}")

sales_channel_revenue = pd.pivot_table(
    df,
    index='sales_channel',
    values='total_claim_amount',
    aggfunc='sum'
)

sales_channel_revenue = sales_channel_revenue.round(2)

print("\n--- 1. Total Revenue by Sales Channel (Pivot) ---")
print(sales_channel_revenue)


clv_by_gender_education = pd.pivot_table(
    df,
    index='gender',
    columns='education',
    values='customer_lifetime_value',
    aggfunc='mean'
)

clv_by_gender_education = clv_by_gender_education.round(2)

print("\n--- 2. avg CLV by Gender and Education (Pivot) ---")
print(clv_by_gender_education)

Dataset 'marketing_customer_analysis_clean.csv' loaded successfully.
--------------------------------------------------

--- 1. Total Revenue by Sales Channel (Pivot Table) ---
               total_claim_amount
sales_channel                    
Agent                  1810226.82
Branch                 1301204.00
Call Center             926600.82
Web                     706600.04

--- 2. Average CLV by Gender and Education (Pivot Table) ---
education  Bachelor  College   Doctor  High School or Below   Master
gender                                                              
F           7874.27  7748.82  7328.51               8675.22  8157.05
M           7703.60  8052.46  7415.33               8149.69  8168.83


1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [ ]:
# Your code goes here